# pyHGT Model Training

In [39]:
!pwd

/home/anushka/FYP/pyHGT-implementation


In [3]:
import mlflow

In [4]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict

In [5]:
import sys
print(sys.executable)

/home/anushka/miniconda3/envs/pyhgt_n/bin/python


In [6]:
sys.path.append('/home/anushka/FYP/pyHGT-implementation')

In [7]:
import matplotlib
print(matplotlib.__version__)
print(matplotlib.__file__)

3.9.4
/home/anushka/miniconda3/envs/pyhgt_n/lib/python3.9/site-packages/matplotlib/__init__.py


In [8]:
import seaborn, inspect
print(seaborn.__version__)
print(seaborn.__file__)

0.13.2
/home/anushka/miniconda3/envs/pyhgt_n/lib/python3.9/site-packages/seaborn/__init__.py


In [9]:
%cd ../
%pwd

/home/anushka/FYP/pyHGT-implementation


'/home/anushka/FYP/pyHGT-implementation'

In [10]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

12.6
True


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

from pyHGT.data import Graph, sample_subgraph, to_torch
from pyHGT.model import GNN, Classifier

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

/home/anushka/miniconda3/envs/pyhgt_n/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [12]:
%pwd

'/home/anushka/FYP/pyHGT-implementation'

In [13]:
# Load the data files

data_dir = 'data/'

patient_tests = pd.read_csv(os.path.join(data_dir,"patient-test.csv"), encoding='latin1')
test_details = pd.read_csv(os.path.join(data_dir,"test-disease-organ.csv"), encoding='latin1')
labels_df = pd.read_csv(os.path.join(data_dir,"patient-one-hot-labeled-disease.csv"), encoding='latin1')

In [14]:
display(patient_tests.head())

,patient_id,report_date,test_name,test_value
0,139760,9/2/2024 6:13,Blood Urea Result,66.0
1,139760,9/2/2024 6:14,ASOT Result,200.0
2,139760,9/2/2024 6:15,ALP Result,168.0
3,200041,10/7/2024 20:59,Fasting Plasma Glucose,83.0
4,200041,10/7/2024 20:59,1 Hr After Plasma Glucose,83.0


In [15]:
display(test_details.head())

,test_name,min,max,organ,organ.1,disease,disease.1,disease.2
0,TSH,0.40,4.00,thyroid,NaN,thyroid disorders,NaN,NaN
1,Fasting Venous Plasma Glucose,100.00,126.00,pancreas,NaN,diabetes mellitus,NaN,NaN
2,Lymphocytes# % Value,20.00,40.00,immune system,NaN,infections,Immune disorders,NaN
3,RDW-SD Absolute Value,39.00,46.00,blood,NaN,anemia,NaN,NaN
4,RBC Absolute Value,4.11,5.51,blood,NaN,anemia,NaN,NaN


In [16]:
display(labels_df.head())

,patient_id,Immune disorders,acid base imbalances,acute,adrenal gland disorders,allergies,anemia,anemia (B12/folate),artharities,asthma,...,leukemia,liver damage,pancreatitis,platelet disorders,prolactinoma,prostate cancer,renal failure,severe dehydration,thrombocytopenia,thyroid disorders
0,139760,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,200041,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,201519,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,201605,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,201839,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
patient_tests["patient_id"] = patient_tests["patient_id"].astype(str)
patient_tests["test_name"] = patient_tests["test_name"].astype(str)

test_details["test_name"] = test_details["test_name"].astype(str)

if "organ" in test_details.columns:
    test_details["organ"] = test_details["organ"].astype(str)

if "disease" in test_details.columns:
    test_details["disease"] = test_details["disease"].astype(str)

test_details = test_details.drop_duplicates(subset=["test_name"]).reset_index(drop=True)

#temporal processing
patient_tests["report_date"] = pd.to_datetime(patient_tests["report_date"])
patient_tests["time_idx"] = patient_tests["report_date"].astype("int64") // 10**9  # convert to unix timestamp in seconds

min_time = patient_tests["time_idx"].min()
patient_tests["rel_time"] = (patient_tests["time_idx"]-min_time)// 86400  # convert to days

max_rel_time = int(patient_tests["rel_time"].max())
print("max_rel_time =", max_rel_time)

max_rel_time = 205


In [18]:
display(patient_tests.head())

,patient_id,report_date,test_name,test_value,time_idx,rel_time
0,139760,2024-09-02 06:13:00,Blood Urea Result,66.0,1725257580,0
1,139760,2024-09-02 06:14:00,ASOT Result,200.0,1725257640,0
2,139760,2024-09-02 06:15:00,ALP Result,168.0,1725257700,0
3,200041,2024-10-07 20:59:00,Fasting Plasma Glucose,83.0,1728334740,36
4,200041,2024-10-07 20:59:00,1 Hr After Plasma Glucose,83.0,1728334740,36


In [19]:
def parse_multi(x):
    if pd.isna(x):
        return []
    x = str(x).strip()

    if ";" in x:
        return [item.strip() for item in x.split(";")]
    
    if "," in x:
        return [item.strip() for item in x.split(",")]
    
    return [x] if x else []


lab_info = {}

for _, row in test_details.iterrows():
    test = str(row["test_name"]).strip()

    organs = parse_multi(row.get("organ", ""))
    diseases = parse_multi(row.get("disease", ""))

    low_th = None
    high_th = None
    if "min" in row and not pd.isna(row["min"]):
        low_th = float(row["min"])
    if "max" in row and not pd.isna(row["max"]):
        high_th = float(row["max"])

    lab_info[test] = {
        "organs": organs,
        "diseases": diseases,
        "low": low_th,
        "high": high_th
    }

print("example lab info:", list(lab_info.items())[:3]) # only show 3 examples

example lab info: [('TSH', {'organs': ['thyroid'], 'diseases': ['thyroid disorders'], 'low': 0.4, 'high': 4.0}), ('Fasting Venous Plasma Glucose', {'organs': ['pancreas'], 'diseases': ['diabetes mellitus'], 'low': 100.0, 'high': 126.0}), ('Lymphocytes#  % Value', {'organs': ['immune system'], 'diseases': ['infections'], 'low': 20.0, 'high': 40.0})]


In [20]:
pt_merged = patient_tests.merge(
    test_details[["test_name", "min", "max"]],
    on="test_name",
    how="left"
)

def compute_abnoramability(row):
    v = float(row["test_value"])
    low = row["min"]
    high = row["max"]

    if pd.isna(low) or pd.isna(high) or low >= high:
        return 0.0  # cannot determine abnormality
    return float((v - low) / (high - low))


pt_merged["abnormality"] = pt_merged.apply(compute_abnoramability, axis=1)

# aggregate per person
agg = pt_merged.groupby("patient_id").agg(
    num_tests = ("test_name", "count"),
    mean_abn = ("abnormality", "mean"),
    max_abn = ("abnormality", "max"),
    min_abn = ("abnormality", "min"),
    last_time = ("rel_time", "max")
).reset_index()

patient_feat_df = agg.set_index("patient_id")
patient_feat_df.head()

,num_tests,mean_abn,max_abn,min_abn,last_time
patient_id,,,,,
1001261,1,0.154722,0.154722,0.154722,131
1001280,3,0.265806,0.477000,0.000000,131
1001398,2,-0.970000,-0.100000,-1.840000,131
1001467,3,0.293917,0.707500,-0.207000,131
1001490,1,3.268182,3.268182,3.268182,132


In [21]:
# extract unique node ids
patient_ids = sorted(patient_tests["patient_id"].astype(str).unique().tolist())
lab_tests = sorted(list(lab_info.keys()))

all_organs = sorted({org for info in lab_info.values() for org in info["organs"] if org})
all_diseases = sorted({dis for info in lab_info.values() for dis in info["diseases"] if dis})

print("#patients =", len(patient_ids))
print("#labs =", len(lab_tests))
print("#organs =", len(all_organs))
print("#diseases =", len(all_diseases))

print("#nodes = ", len(patient_ids) + len(lab_tests) + len(all_organs) + len(all_diseases))

#patients = 24352
#labs = 172
#organs = 19
#diseases = 45
#nodes =  24588


In [22]:
#build pyHGT graph - add nodes

graph = Graph()

for pid in patient_ids:
    if pid in patient_feat_df.index:
        row = patient_feat_df.loc[pid]
        node = {
            "type": "patient",
            "id": pid,
            "num_tests": float(row["num_tests"]),
            "mean_abn":  float(row["mean_abn"]),
            "max_abn":   float(row["max_abn"]),
            "min_abn":   float(row["min_abn"]),
            "last_time": int(row["last_time"]),
            "time":      int(row["last_time"]),
        }
    
    else:
        node = {
            "type": "patient",
            "id": pid,
            "num_tests": 0.0,
            "mean_abn":  0.0,
            "max_abn":   0.0,
            "min_abn":   0.0,
            "last_time": 0,
            "time":      0,
        }
    graph.add_node(node)

for test in lab_tests:
    info = lab_info[test]
    graph.add_node({
        "type": "lab_test",
        "id": test,
        "time": 0,
        "low": 0.0 if info["low"] is None else float(info["low"]),
        "high": 0.0 if info["high"] is None else float(info["high"])
    })


for organ in all_organs:
    graph.add_node({
        "type": "organ",
        "id": organ,
        "time": 0
    })

for disease in all_diseases:
    graph.add_node({
        "type": "disease",
        "id": disease,
        "time": 0
    })

In [23]:
# Convenience maps from original ID -> internal index
patient2idx = graph.node_forward["patient"]
lab2idx     = graph.node_forward["lab_test"]
organ2idx   = graph.node_forward["organ"]
disease2idx = graph.node_forward["disease"]

print("Example patient2idx:", list(lab2idx.items())[:5])

Example patient2idx: [('1 Hr After Plasma Glucose', 0), ('2 Hr After Breakfast Plasma Glucose', 1), ('2 Hrs After Dinner Plasma Glucose', 2), ('2 Hrs After LunchPlasma Glucose', 3), ('2 Hrs After Plasma Glucose', 4)]


In [24]:
#build pyHGT graph - add edges with temporal info

for _, row in patient_tests.iterrows():
    pid = str(row["patient_id"])
    test = str(row["test_name"])
    t = int(row["rel_time"])

    if pid in patient2idx and test in lab2idx: # src and dst nodes exist
        # patient - lab_test edge
        graph.add_edge(
            {"type": "patient", "id": pid},
            {"type": "lab_test", "id": test},
            time=t,
            relation_type="had_test",
            directed=True,
        )

for test, info in lab_info.items():

    # lab_test - organ edges
    for organ in info["organs"]:
        if organ in organ2idx:
            graph.add_edge(
                {"type": "lab_test", "id": test},
                {"type": "organ", "id": organ},
                relation_type="tests_organ",
                directed=True,
                time=0
            )

    # lab_test - disease edges
    for disease in info["diseases"]:
        if disease in disease2idx:
            graph.add_edge(
                {"type": "lab_test", "id": test},
                {"type": "disease", "id": disease},
                relation_type="associated_with",
                directed=True,
                time=0
            )

            for organ in info["organs"]:
                if organ in organ2idx:
                    graph.add_edge(
                        {"type": "disease", "id": disease},
                        {"type": "organ", "id": organ},
                        relation_type="occurs_in",
                        directed=True,
                        time=0
                    )

print("Meta relations in graph:", graph.get_meta_graph())

Meta relations in graph: [('lab_test', 'patient', 'had_test'), ('lab_test', 'organ', 'rev_tests_organ'), ('lab_test', 'disease', 'rev_associated_with'), ('patient', 'lab_test', 'rev_had_test'), ('organ', 'lab_test', 'tests_organ'), ('organ', 'disease', 'occurs_in'), ('disease', 'lab_test', 'associated_with'), ('disease', 'organ', 'rev_occurs_in')]


In [25]:
#node features

for t, node_list in graph.node_bacward.items():
    df = pd.DataFrame(node_list).reset_index(drop=True)
    graph.node_feature[t] = df

for t,df in graph.node_feature.items():
    print(f"Node type: {t}, feature shape: {df.shape}")
    display(df.head())

Node type: patient, feature shape: (24352, 8)


,type,id,num_tests,mean_abn,max_abn,min_abn,last_time,time
0,patient,1001261,1.0,0.154722,0.154722,0.154722,131,131
1,patient,1001280,3.0,0.265806,0.477000,0.000000,131,131
2,patient,1001398,2.0,-0.970000,-0.100000,-1.840000,131,131
3,patient,1001467,3.0,0.293917,0.707500,-0.207000,131,131
4,patient,1001490,1.0,3.268182,3.268182,3.268182,132,132


Node type: lab_test, feature shape: (172, 5)


,type,id,time,low,high
0,lab_test,1 Hr After Plasma Glucose,0,0.0,180.0
1,lab_test,2 Hr After Breakfast Plasma Glucose,0,0.0,140.0
2,lab_test,2 Hrs After Dinner Plasma Glucose,0,0.0,140.0
3,lab_test,2 Hrs After LunchPlasma Glucose,0,0.0,140.0
4,lab_test,2 Hrs After Plasma Glucose,0,0.0,140.0


Node type: organ, feature shape: (19, 3)


,type,id,time
0,organ,Immune system,0
1,organ,Kidney,0
2,organ,blood,0
3,organ,bone narrow,0
4,organ,cardiovascular system,0


Node type: disease, feature shape: (45, 3)


,type,id,time
0,disease,acute,0
1,disease,adrenal gland disorders,0
2,disease,allergies,0
3,disease,anemia,0
4,disease,anemia (B12/folate),0


In [26]:
# feature_medical extractor for sample subgraph

def feature_medical(layer_data, graph):
    feature = {}
    times = {}
    indxs = {}
    texts = {}

    # for t in layer_data:
    #     if len(layer_data[t]) == 0:
    #         continue

    all_types = graph.node_feature.keys()

    for t in layer_data:
        if len(layer_data[t]) == 0:
            # feature[t] = np.zeros((0,5), dtype=np.float32)  # empty features
            # times[t] = np.array((0,), dtype=np.int32)
            # indxs[t] = np.array((0,), dtype=np.int32)
            continue

        idxs = np.array(list(layer_data[t].keys()))
        tims = np.array(list(layer_data[t].values()))[:,1]

        df = graph.node_feature[t]
        feats = np.zeros((len(idxs),5), dtype=np.float32)  # 5 features per node

        if t == "patient":

            cols = ["num_tests", "mean_abn", "max_abn", "min_abn", "last_time"]
            vals = df.loc[idxs, cols].fillna(0).values.astype(np.float32)
            feats = vals # all 5 features

        elif t == "lab_test":
            for c in ["low", "high"]:
                if c not in df.columns:
                    df[c] = 0.0
            vals = df.loc[idxs, ["low", "high"]].fillna(0).values.astype(np.float32)
            feats[:,0:2] = vals  # first 2 features and other 3 are zeros

        else:
            pass  # no features for organ and disease nodes

        feature[t] = feats
        times[t] = tims
        indxs[t] = idxs

    return feature, times, indxs, texts    

In [32]:
#subgraph sampling test

from pyHGT.data import sample_subgraph, to_torch

all_patient_indices = np.array(list(graph.node_forward["patient"].values()))
batch_pids = all_patient_indices[:8] # first 8 patients for testing

inp = {
    "patient": [(int(pid),0) for pid in batch_pids]
}

feature, times, edge_list, indxs, texts = sample_subgraph(
    graph,
    time_range={max_rel_time: True},
    sampled_depth=2,
    sampled_number=8,
    inp = inp,
    feature_extractor=feature_medical
)

node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = to_torch(
    feature,times,edge_list, graph
)

/home/anushka/FYP/pyHGT-implementation/pyHGT/data.py:251: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  node_feature = torch.FloatTensor(node_feature)


In [33]:
print("node_feature:", node_feature.shape)
print("node_type:", node_type.shape)
print("edge_index:", edge_index.shape)
print("edge_time:", edge_time.shape)
print("edge_type:", edge_type.shape)
print("node_dict:", node_dict)
print("edge_dict:", edge_dict)

node_feature: torch.Size([46, 5])
node_type: torch.Size([46])
edge_index: torch.Size([2, 222])
edge_time: torch.Size([222])
edge_type: torch.Size([222])
node_dict: {'patient': [0, 0], 'lab_test': [16, 1], 'organ': [32, 2], 'disease': [38, 3]}
edge_dict: {'had_test': 0, 'rev_tests_organ': 1, 'rev_associated_with': 2, 'rev_had_test': 3, 'tests_organ': 4, 'occurs_in': 5, 'associated_with': 6, 'rev_occurs_in': 7, 'self': 8}


### Label Define

In [34]:
disease_label_cols = [c for c in labels_df.columns if c != "patient_id"]
num_diseases = len(disease_label_cols)

print("Label columns:", disease_label_cols)
print("Number of diseases to predict:", num_diseases)

Label columns: ['Immune disorders', 'acid base imbalances', 'acute', 'adrenal gland disorders', 'allergies', 'anemia', 'anemia (B12/folate)', 'artharities', 'asthma', 'biliary obstruction', 'cancer', 'cardiovascular diseases', 'cholestrol', 'chronic Liver disease', 'chronic infections', 'chronic kidney disease', 'chronic liver disease', 'cirrhosis', 'dehydration', 'diabetes', 'diabetes insipidus', 'diabetes mellitus', 'fertility conditions', 'fluid', 'heart arithmias', 'heart attacks', 'hemolytic anemia', 'high blood pressure', 'hyperlipidaemia', 'hypopituitarism', 'infections', 'inflammation', 'kidney damage', 'kidney stones', 'leukemia', 'liver damage', 'pancreatitis', 'platelet disorders', 'prolactinoma', 'prostate cancer', 'renal failure', 'severe dehydration', 'thrombocytopenia', 'thyroid disorders']
Number of diseases to predict: 44


In [35]:
#torch tensor for labels

num_patients_in_graph = len(patient2idx)
Y = torch.zeros((num_patients_in_graph, num_diseases), dtype=torch.float32)

for _, row in labels_df.iterrows():
    pid = str(row["patient_id"])
    if pid in patient2idx:
        idx = patient2idx[pid]
        Y[idx] = torch.tensor(row[disease_label_cols].values, dtype=torch.float32)

print("Labels tensor shape:", Y.shape)

Labels tensor shape: torch.Size([24352, 44])


### Training

In [36]:
types = graph.get_types()
num_types = len(types)

meta_rels = graph.get_meta_graph()
num_relations = len(meta_rels) + 1

print("Node types:", types)
print("Number of node types:", num_types)
print("Meta relations:", meta_rels)
print("Number of relations:", num_relations)

Node types: ['patient', 'lab_test', 'organ', 'disease']
Number of node types: 4
Meta relations: [('lab_test', 'patient', 'had_test'), ('lab_test', 'organ', 'rev_tests_organ'), ('lab_test', 'disease', 'rev_associated_with'), ('patient', 'lab_test', 'rev_had_test'), ('organ', 'lab_test', 'tests_organ'), ('organ', 'disease', 'occurs_in'), ('disease', 'lab_test', 'associated_with'), ('disease', 'organ', 'rev_occurs_in')]
Number of relations: 9


In [37]:
in_dim = 5
hidden_dim = 256
n_heads = 8
n_layers  =2
dropout   = 0.2

num_diseases = Y.shape[1]  # number of disease classes

gnn = GNN(
    in_dim=in_dim,
    n_hid=hidden_dim,
    num_types=num_types,
    num_relations=num_relations,
    n_heads=n_heads,
    n_layers=n_layers,
    dropout=dropout,
    conv_name='hgt',
    prev_norm=False,
    last_norm=False,
    use_RTE=True
).to(device)

## Multi-label classifier
class MultilabelClassifier(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.linear = nn.Linear(in_dim, out_dim)

    def forward(self, x):
        return self.linear(x)
    
clf = MultilabelClassifier(hidden_dim, num_diseases).to(device)

params = list(gnn.parameters()) + list(clf.parameters())

optimizer = optim.Adam(params, lr=0.001, weight_decay=1e-5)

# BCEWithLogitsLoss for multi-label classification (multiple diseases per patient)
criterion = nn.BCEWithLogitsLoss()

In [38]:
# Train/test/val split

all_patient_indices = np.array(list(graph.node_forward["patient"].values())) #defined above also. incase of errors remove this line if needed
np.random.shuffle(all_patient_indices)

N = len(all_patient_indices)
n_train = int(0.7 * N)
n_val   = int(0.15 * N)

train_idx = all_patient_indices[:n_train]
val_idx   = all_patient_indices[n_train:n_train+n_val]
test_idx  = all_patient_indices[n_train+n_val:]

print("#train =", len(train_idx))
print("#val   =", len(val_idx))
print("#test  =", len(test_idx))


#train = 17046
#val   = 3652
#test  = 3654


In [39]:
def get_patient_batch_embeddings(batch_pids, graph, time_range, sampled_depth=2, sampled_number=8):
    
    inp = {
        "patient": [(int(pid),0) for pid in batch_pids]
    }

    feature, times, edge_list, indxs, texts = sample_subgraph(
        graph,
        time_range={max_rel_time: True},
        sampled_depth=2,
        sampled_number=8,
        inp = inp,
        feature_extractor=feature_medical
    )

    node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = to_torch(
        feature,times,edge_list, graph
    )

    node_feature = node_feature.to(device)
    node_type = node_type.to(device)
    edge_time = edge_time.to(device)
    edge_index = edge_index.to(device)
    edge_type = edge_type.to(device)

    with torch.set_grad_enabled(True):
        all_embs = gnn(node_feature, node_type, edge_time, edge_index, edge_type)

    patient_offset, patient_type_id =node_dict["patient"]

    local_patient_ids = indxs["patient"]
    pid_to_local = {int(pid): i for i, pid in enumerate(local_patient_ids)}


    selected_global_indices = []
    for pid in batch_pids:
        if pid in pid_to_local:
            local_id = pid_to_local[pid]
            global_node_idx = patient_offset + local_id
            selected_global_indices.append(global_node_idx)

        else:
            pass

    if len(selected_global_indices) == 0:

        return None, None
    
    selected_global_indices = torch.LongTensor(selected_global_indices).to(device)
    batch_embs = all_embs[selected_global_indices]

    batch_labels = Y[batch_pids]
    batch_labels = batch_labels.to(device)

    return batch_embs, batch_labels

In [40]:
#time range define
from sklearn.metrics import f1_score, accuracy_score


time_range={max_rel_time: True}

#Train/val loop

def evaluate(split_idx, graph, time_range, batch_size=64):
    gnn.eval()
    clf.eval()
    losses = []
    all_preds = []
    all_trues = []

    with torch.no_grad():
        for i in range(0, len(split_idx), batch_size):
            batch_pids = split_idx[i:i+batch_size]
            batch_embs, batch_labels = get_patient_batch_embeddings(
                batch_pids, graph, time_range
            )

            if batch_embs is None:
                continue

            logits = clf(batch_embs)
            loss = criterion(logits, batch_labels)

            losses.append(loss.item())
            trues = batch_labels.cpu().numpy()

            all_preds.append(torch.sigmoid(logits).cpu().numpy())
            all_trues.append(batch_labels.cpu().numpy())
    
    if len(losses) == 0:
        return None, None, None
    

    all_preds = np.vstack(all_preds)
    all_trues = np.vstack(all_trues)

    # Threshold predictions at 0.5 to convert to binary for F1 score and accuracy
    all_preds_binary = (all_preds > 0.5).astype(int)

    f1 = f1_score(all_trues.flatten(), all_preds_binary.flatten(), zero_division=0)
    accuracy = accuracy_score(all_trues.flatten(), all_preds_binary.flatten())

    return float(np.mean(losses)), float(f1), float(accuracy)

In [ ]:
#Full training loop

hyperParams = {
    "num_epochs": 50,
    "batch_size":  128,
    "sampled_depth": 2,
    "sampled_number": 8,
    "learning_rate": 0.001,
    "weight_decay": 1e-5
}

# Set MLflow experiment
mlflow.set_experiment("pyHGT-Disease-Prediction")

with mlflow.start_run(run_name=f"pyHGT_epochs{hyperParams['num_epochs']}_batch{hyperParams['batch_size']}"):

    # Log hyperparameters
    mlflow.log_params(hyperParams)
    
    # Log model architecture details
    mlflow.log_params({
        "in_dim": in_dim,
        "hidden_dim": hidden_dim,
        "n_heads": n_heads,
        "n_layers": n_layers,
        "dropout": dropout,
        "num_diseases": num_diseases,
        "num_types": num_types,
        "num_relations": num_relations
    })

    train_idx_arr = np.array(train_idx)
    val_idx_arr = np.array(val_idx)
    test_idx_arr = np.array(test_idx)

    train_losses = []
    train_acc = []
    val_losses = []
    val_acc = []
    val_f1s = []

    for epoch in range(1, hyperParams["num_epochs"]+1):
        gnn.train()
        clf.train()

        perm = np.random.permutation(len(train_idx_arr))
        train_idx_arr_shuffled = train_idx_arr[perm]

        epoch_losses = []
        epoch_preds = []
        epoch_trues = []

        for i in range(0, len(train_idx_arr), hyperParams["batch_size"]):
            batch_pids = train_idx_arr_shuffled[i:i+hyperParams["batch_size"]]

            optimizer.zero_grad()

            batch_embs, batch_labels = get_patient_batch_embeddings(
                batch_pids, graph, time_range,
                sampled_depth=hyperParams["sampled_depth"],
                sampled_number=hyperParams["sampled_number"]
            )

            if batch_embs is None:
                continue
            
            logits = clf(batch_embs)
            loss = criterion(logits, batch_labels)
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())
            
            # Collect predictions and labels for training accuracy
            with torch.no_grad():
                preds = torch.sigmoid(logits).cpu().numpy()
                trues = batch_labels.cpu().numpy()
                epoch_preds.append(preds)
                epoch_trues.append(trues)

        train_loss = np.mean(epoch_losses) if epoch_losses else None
        
        # Calculate training accuracy
        if len(epoch_preds) > 0:
            all_train_preds = np.vstack(epoch_preds)
            all_train_trues = np.vstack(epoch_trues)
            train_preds_binary = (all_train_preds > 0.5).astype(int)
            train_accuracy = accuracy_score(all_train_trues.flatten(), train_preds_binary.flatten())
        else:
            train_accuracy = None
        
        val_loss, val_f1, val_accuracy = evaluate(val_idx_arr, graph, time_range, hyperParams["batch_size"])
        
        train_losses.append(train_loss)
        train_acc.append(train_accuracy)
        val_losses.append(val_loss)
        val_acc.append(val_accuracy)
        val_f1s.append(val_f1)

        # Log metrics to MLflow
        mlflow.log_metrics({
            "train_loss": train_loss,
            "train_accuracy": train_accuracy,
            "val_loss": val_loss,
            "val_accuracy": val_accuracy,
            "val_f1": val_f1
        }, step=epoch)

        print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Train Acc = {train_accuracy:.4f}, Val Loss = {val_loss:.4f}, Val Acc = {val_accuracy:.4f}, Val F1 = {val_f1:.4f}")

    # Evaluate on test set
    print("\nEvaluating on test set...")
    test_loss, test_f1, test_accuracy = evaluate(test_idx_arr, graph, time_range, batch_size=hyperParams["batch_size"])
    print(f"Test Loss = {test_loss:.4f}, Test Acc = {test_accuracy:.4f}, Test F1 = {test_f1:.4f}")
    
    # Log final test metrics
    mlflow.log_metrics({
        "test_loss": test_loss,
        "test_accuracy": test_accuracy,
        "test_f1": test_f1
    })
    
    # Log models to MLflow
    mlflow.pytorch.log_model(gnn, "gnn_model")
    mlflow.pytorch.log_model(clf, "classifier_model")
    
    print(f"\nMLflow run completed. Run ID: {mlflow.active_run().info.run_id}")

In [ ]:
#Draw training/validation loss curves
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.plot(range(1, hyperParams["num_epochs"]+1), train_losses, label='Train Loss')
plt.plot(range(1, hyperParams["num_epochs"]+1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss over Epochs')
plt.legend()
plt.savefig('loss_curve.png', dpi=150, bbox_inches='tight')
plt.show()

# Log plot to MLflow if run is active
try:
    mlflow.log_artifact('loss_curve.png')
except:
    print("No active MLflow run to log artifact")

In [ ]:
#Draw F1 score and accuracy curves
plt.figure(figsize=(15,5))

# F1 Score
plt.subplot(1, 2, 1)
plt.plot(range(1, hyperParams["num_epochs"]+1), val_f1s, label='Validation F1 Score', color='orange')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('Validation F1 Score over Epochs')
plt.legend()
plt.grid(True, alpha=0.3)

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, hyperParams["num_epochs"]+1), train_acc, label='Train Accuracy', color='blue')
plt.plot(range(1, hyperParams["num_epochs"]+1), val_acc, label='Validation Accuracy', color='green')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy over Epochs')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('metrics_curves.png', dpi=150, bbox_inches='tight')
plt.show()

# Log plot to MLflow if run is active
try:
    mlflow.log_artifact('metrics_curves.png')
except:
    print("No active MLflow run to log artifact")

### Testing and Testing Loss

In [ ]:
# Note: Test evaluation is now performed inside the MLflow run context in the training loop
# This cell can be used for re-evaluation if needed
test_loss, test_f1, test_accuracy = evaluate(test_idx_arr, graph, time_range, batch_size=128)
print(f"Test Loss = {test_loss:.4f}, Test Acc = {test_accuracy:.4f}, Test F1 = {test_f1:.4f}")

In [ ]:
# Save the trained model
torch.save({
    'gnn_state_dict': gnn.state_dict(),
    'clf_state_dict': clf.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, 'trained_model3.pth')

## Testing and Inferencing

In [41]:
# Count model parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

gnn_params = count_parameters(gnn)
clf_params = count_parameters(clf)
total_params = gnn_params + clf_params

print(f"GNN parameters: {gnn_params:,}")
print(f"Classifier parameters: {clf_params:,}")
print(f"Total trainable parameters: {total_params:,}")

GNN parameters: 2,794,136
Classifier parameters: 11,308
Total trainable parameters: 2,805,444


In [43]:
import torch
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

gnn.load_state_dict(torch.load('trained_model3.pth')['gnn_state_dict'])
clf.load_state_dict(torch.load('trained_model3.pth')['clf_state_dict'])

gnn.to(device).eval()
clf.to(device).eval()

print(f"Models loaded on to {device} and set to evaluation mode.")

Models loaded on to cuda and set to evaluation mode.


In [44]:
#Helper utilities
def normalize_lab_value(val, low, high, eps=1e-6):
    return (float(val) - float(low)) / (float(high) - float(low) + eps)

def print_top_diseases(probs, disease_names, k =10, min_prob=0.0):
    probs = probs.detach().cpu().numpy().astype(float)
    idx = np.argsort(-probs)[:k]
    out= []
    for i in idx:
        if probs[i] >= min_prob:
            out.append((disease_names[i], float(probs[i])))
    return out

In [45]:
from pyHGT.data import sample_subgraph, to_torch

def predict_patient_diseases_from_graph(graph, seed_patient_idx, max_time, sampled_depth=2, sampled_number=16):

    inp = {"patient" : [(int(seed_patient_idx), int(max_time))]}

    feature, times, edge_list, indxs, texts = sample_subgraph(
        graph,
        time_range={int(max_time): True},
        sampled_depth=sampled_depth,
        sampled_number=sampled_number,
        inp = inp,
        feature_extractor=feature_medical
    )

    node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = to_torch(
        feature,times,edge_list, graph
    )

    node_feature = node_feature.to(device)
    node_type = node_type.to(device)
    edge_time = edge_time.to(device)
    edge_index = edge_index.to(device)
    edge_type = edge_type.to(device)

    with torch.no_grad():
        h = gnn(node_feature, node_type, edge_time, edge_index, edge_type)

        p_offset, p_type_id = node_dict["patient"]

        patient_mask = (node_type == p_type_id)
        patient_embs = h[patient_mask]

        logits = clf(patient_embs)
        probs = torch.sigmoid(logits).squeeze(0)

    return probs

### Testing on a known patient

In [46]:
pid_str = "1093819"
seed_idx = graph.node_forward["patient"][pid_str]

probs = predict_patient_diseases_from_graph(
    graph,
    seed_idx,
    max_time=max_rel_time,
)

top = print_top_diseases(probs, disease_label_cols, k=10, min_prob=0.1)
print("Top predicted disease for patient", pid_str)
for d, p in top:
    print(f"{d:40s} {p:.3f}")

IndexError: index 17 is out of bounds for axis 0 with size 17